### Import libraries

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
from skimage import io
import sys
import seaborn as sns
from scipy import spatial

In [2]:
sys.path.insert(1, "/Users/k1801626/OneDrive - King's College London/git/AFT-Alignment_by_Fourier_Transform/Python_implementation/")
import AFT_tools as AFT       

In [88]:
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

### Import data (csv tracks and images)

In [3]:
csv_folder = "/Users/k1801626/OneDrive - King's College London/data/JP/AFT_track/"
image_folder = "/Users/k1801626/OneDrive - King's College London/data/JP/AFT_track/images/"

In [5]:
spots_list = glob.glob(csv_folder+'*Spots.csv')

print(*spots_list, sep="\n")

/Users/k1801626/OneDrive - King's College London/data/JP/AFT_track/merged_Spots.csv


In [6]:
tracks_list = glob.glob(csv_folder+'*Tracks.csv')

print(*tracks_list, sep="\n")

/Users/k1801626/OneDrive - King's College London/data/JP/AFT_track/merged_Tracks.csv


In [10]:
image_list = glob.glob(image_folder+'*.tif')

print(*image_list, sep="\n")

/Users/k1801626/OneDrive - King's College London/data/JP/AFT_track/images/Position_6_AFT.tif
/Users/k1801626/OneDrive - King's College London/data/JP/AFT_track/images/Position_12_AFT.tif


### Parameters

In [104]:
output_image_folder = csv_folder+'output_images/'

In [38]:
single_frame = False

In [12]:
#### required AFT parameters ####
window_size = 100
overlap = 0.6
neighborhood_radius = 5

### Load data

In [14]:
df_spots = pd.read_csv(spots_list[0])
df_spots.head()

,LABEL,ID,TRACK_ID,QUALITY,POSITION_X,POSITION_Y,POSITION_Z,POSITION_T,FRAME,RADIUS,...,PERIMETER,CIRCULARITY,SOLIDITY,SHAPE_INDEX,File_name_raw,Condition,experiment_nb,File_name,Repeat,Unique_ID
0,ID2560,2560,1,113788.0,446.481612,463.926373,0.0,26.0,26,190.336445,...,1850.608931,0.417613,0.884730,5.485520,Position_26,siCCT8_si5,R1,siCCT8_si5_R1_Position_26,1,siCCT8_si5_R1_Position_26_1
1,ID2435,2435,1,64912.0,186.902997,251.383041,0.0,1.0,1,143.727227,...,1103.639245,0.669551,0.961309,4.332246,Position_26,siCCT8_si5,R1,siCCT8_si5_R1_Position_26,1,siCCT8_si5_R1_Position_26_1
2,ID2563,2563,1,111268.0,432.193334,454.720643,0.0,27.0,27,188.187463,...,1845.354851,0.410564,0.845194,5.532409,Position_26,siCCT8_si5,R1,siCCT8_si5_R1_Position_26,1,siCCT8_si5_R1_Position_26_1
3,ID2437,2437,1,71143.0,230.466989,274.844695,0.0,4.0,4,150.484286,...,1161.466995,0.662717,0.959538,4.354525,Position_26,siCCT8_si5,R1,siCCT8_si5_R1_Position_26,1,siCCT8_si5_R1_Position_26_1
4,ID2565,2565,1,123560.0,441.649200,460.855612,0.0,28.0,28,198.340124,...,1885.321437,0.436928,0.854008,5.362902,Position_26,siCCT8_si5,R1,siCCT8_si5_R1_Position_26,1,siCCT8_si5_R1_Position_26_1


In [15]:
df_tracks = pd.read_csv(tracks_list[0])
df_tracks.head()

,LABEL,TRACK_INDEX,TRACK_ID,DIVISION_TIME_MEAN,DIVISION_TIME_STD,NUMBER_SPOTS,NUMBER_GAPS,NUMBER_SPLITS,NUMBER_MERGES,NUMBER_COMPLEX,...,CONFINEMENT_RATIO,MEAN_STRAIGHT_LINE_SPEED,LINEARITY_OF_FORWARD_PROGRESSION,MEAN_DIRECTIONAL_CHANGE_RATE,File_name_raw,Condition,experiment_nb,File_name,Repeat,Unique_ID
0,Track_1,1,1,NaN,NaN,76,0,0,0,0,...,0.651187,10.556033,0.651187,1.235458,Position_26,siCCT8_si5,R1,siCCT8_si5_R1_Position_26,1,siCCT8_si5_R1_Position_26_1
1,Track_2,2,2,NaN,NaN,55,0,0,0,0,...,0.013550,0.361223,0.013550,1.681933,Position_26,siCCT8_si5,R1,siCCT8_si5_R1_Position_26,1,siCCT8_si5_R1_Position_26_2
2,Track_3,3,3,NaN,NaN,76,0,0,0,0,...,0.149645,1.959934,0.149645,1.803235,Position_26,siCCT8_si5,R1,siCCT8_si5_R1_Position_26,1,siCCT8_si5_R1_Position_26_3
3,Track_4,4,4,NaN,NaN,76,0,0,0,0,...,0.044778,0.713082,0.044778,1.874086,Position_26,siCCT8_si5,R1,siCCT8_si5_R1_Position_26,1,siCCT8_si5_R1_Position_26_4
4,Track_8,8,8,NaN,NaN,29,0,0,0,0,...,0.140809,2.646769,0.140809,1.866365,Position_26,siCCT8_si5,R1,siCCT8_si5_R1_Position_26,1,siCCT8_si5_R1_Position_26_8


In [16]:
movie_list = df_spots.File_name_raw.unique()
print(*movie_list, sep="\n")

Position_26
Position_27
Position_31
Position_33
Position_6
Position_12


### Find images for current experimental condition

In [35]:
im_list_current = []
for im_file in range(len(image_list)):
    temp_im_file = image_list[im_file].split("/")[-1].split("_",2)[:2]
    temp_im_file = '_'.join(temp_im_file)
    im_list_current = np.append(im_list_current, temp_im_file)

print(im_list_current)

['Position_6' 'Position_12']


### Run analysis

In [81]:
df_subset_out = pd.DataFrame()
for position in range(len(im_list_current)):

    # load current image
    im = io.imread(image_list[position])
    if single_frame == True:
        im = im[0,]

    # run AFT on all (relevant) frames
    x, y, u, v, im_theta, im_eccentricity = AFT.image_local_order(im, window_size, overlap,
                                                                  plot_overlay=False, 
                                                                  plot_angles=False,
                                                                  plot_eccentricity=False,
                                                                  save_figures=False)
    # get coords for AFT grid
    AFT_coords = np.empty(shape=(len(x),2))
    AFT_coords[:,0] = x
    AFT_coords[:,1] = y

    # get relevant part of spots csv
    df_subset = df_spots.loc[df_spots.File_name_raw == im_list_current[position]]

    # initialise df for ouput of track analysis
    df_track_out = pd.DataFrame(columns = ['TRACK_ID','FRAME','track_angle','AFT_angle','AFT_track_angle','AFT_track_angle_cos2',
                                    'track_angle_u', 'track_angle_v','AFT_angle_u','AFT_angle_v'])

    # for each track
    for trackID in range(len(df_subset.TRACK_ID.unique())):
    
        # create df for current track
        df_track = df_subset.loc[df_subset.TRACK_ID == df_subset.TRACK_ID.unique()[trackID]]
        df_track = df_track.sort_values(by='FRAME')
        df_track = df_track.reset_index(drop=True)
        df_track = df_track.drop_duplicates(subset=['FRAME'], ignore_index=True)
    
        # initialise ouput arrays
        AFT_angle = []
        track_angle = []
        current_time_point = []
    
        # calculate angles for AFT and track
        if single_frame == False:
            for time_point in range(len(df_track)-1):
                
                track_current = [df_track.loc[time_point, 'POSITION_X'], df_track.loc[time_point, 'POSITION_Y']]
                track_next = [df_track.loc[time_point+1, 'POSITION_X'], df_track.loc[time_point+1, 'POSITION_Y']]
                
                d_closest,idx_closest = spatial.KDTree(AFT_coords).query(track_current)
                AFT_angle = np.append(AFT_angle, np.ravel(im_theta[time_point])[idx_closest])
                
                track_length = [track_next[0]-track_current[0], track_next[1]-track_current[1]]
                track_norm = np.sqrt(track_length[0] ** 2 + track_length[1] ** 2)
                track_direction = [track_length[0]/track_norm, track_length[1]/track_norm]
                track_angle = np.append(track_angle, np.arctan2(track_direction[1], track_direction[0]))
            
                current_time_point = np.append(current_time_point, time_point)
        else:
            for time_point in range(len(df_track)-1):
                
                track_current = [df_track.loc[time_point, 'POSITION_X'], df_track.loc[time_point, 'POSITION_Y']]
                track_next = [df_track.loc[time_point+1, 'POSITION_X'], df_track.loc[time_point+1, 'POSITION_Y']]
                
                d_closest,idx_closest = spatial.KDTree(AFT_coords).query(track_current)
                AFT_angle = np.append(AFT_angle, np.ravel(im_theta)[idx_closest])
                
                track_length = [track_next[0]-track_current[0], track_next[1]-track_current[1]]
                track_norm = np.sqrt(track_length[0] ** 2 + track_length[1] ** 2)
                track_direction = [track_length[0]/track_norm, track_length[1]/track_norm]
                track_angle = np.append(track_angle, np.arctan2(track_direction[1], track_direction[0]))
            
                current_time_point = np.append(current_time_point, time_point)
    
        # calculate angle difference and cosine squared of the angle difference
        AFT_track_angle = AFT_angle-track_angle
        AFT_track_angle_cos2 = np.cos(AFT_track_angle) ** 2

        # create temporary df
        df_out = pd.DataFrame(columns = ['TRACK_ID','FRAME','track_angle','AFT_angle','AFT_track_angle','AFT_track_angle_cos2',
                                    'track_angle_u', 'track_angle_v','AFT_angle_u','AFT_angle_v'])
    
        df_out.TRACK_ID = np.full((len(df_track)-1, ), df_subset.TRACK_ID.unique()[trackID])
        df_out.FRAME = current_time_point
        df_out.track_angle = track_angle
        df_out.AFT_angle = AFT_angle
        df_out.AFT_track_angle = AFT_track_angle
        df_out.AFT_track_angle_cos2 = AFT_track_angle_cos2
        
        df_out.track_angle_u = np.cos(track_angle)
        df_out.track_angle_v = np.sin(track_angle)
        df_out.AFT_angle_u = np.cos(AFT_angle)
        df_out.AFT_angle_v = np.sin(AFT_angle)

        df_out_merge = pd.merge(df_track, df_out, on=['TRACK_ID','FRAME'], how='outer')

        # concatenate output df
        df_list = [df_track_out, df_out_merge]
        df_track_out = pd.concat([df_track_out for df_track_out in df_list if not df_track_out.empty])

    df_list1 = [df_subset_out, df_track_out]
    df_subset_out = pd.concat([df_subset_out for df_subset_out in df_list1 if not df_subset_out.empty])

/var/folders/b8/814qyc5s5jx004x1r459c2gc0000gs/T/ipykernel_72169/169628012.py:53: RuntimeWarning: invalid value encountered in scalar divide
  track_direction = [track_length[0]/track_norm, track_length[1]/track_norm]
/var/folders/b8/814qyc5s5jx004x1r459c2gc0000gs/T/ipykernel_72169/169628012.py:53: RuntimeWarning: invalid value encountered in scalar divide
  track_direction = [track_length[0]/track_norm, track_length[1]/track_norm]
/var/folders/b8/814qyc5s5jx004x1r459c2gc0000gs/T/ipykernel_72169/169628012.py:53: RuntimeWarning: invalid value encountered in scalar divide
  track_direction = [track_length[0]/track_norm, track_length[1]/track_norm]


### Plot

In [91]:
cmap = plt.get_cmap('hsv')
position = 0
# load current image
im = io.imread(image_list[position])
if single_frame == True:
    im = im[0,]

# run AFT on all (relevant) frames
x, y, u, v, im_theta, im_eccentricity = AFT.image_local_order(im, window_size, overlap, save_path = [],
                                                              plot_overlay=False, 
                                                              plot_angles=False,
                                                              plot_eccentricity=False,
                                                              save_figures=False)
# get relevant part of subset_out df
df_temp = df_subset_out.loc[df_subset_out.File_name_raw == im_list_current[position]]

In [93]:
df_temp.loc[df_temp.FRAME == 0]

,LABEL,ID,TRACK_ID,QUALITY,POSITION_X,POSITION_Y,POSITION_Z,POSITION_T,FRAME,RADIUS,VISIBILITY,MANUAL_SPOT_COLOR,MEAN_INTENSITY_CH1,MEDIAN_INTENSITY_CH1,MIN_INTENSITY_CH1,MAX_INTENSITY_CH1,TOTAL_INTENSITY_CH1,STD_INTENSITY_CH1,CONTRAST_CH1,SNR_CH1,ELLIPSE_X0,ELLIPSE_Y0,ELLIPSE_MAJOR,ELLIPSE_MINOR,ELLIPSE_THETA,ELLIPSE_ASPECTRATIO,AREA,PERIMETER,CIRCULARITY,SOLIDITY,SHAPE_INDEX,File_name_raw,Condition,experiment_nb,File_name,Repeat,Unique_ID,track_angle,AFT_angle,AFT_track_angle,AFT_track_angle_cos2,track_angle_u,track_angle_v,AFT_angle_u,AFT_angle_v
0,ID3413,3413,0,60629.0,911.573447,1229.346787,0.0,0.0,0,138.891513,1,NaN,4.994885,5.0,0.0,5.0,302695.0,0.159848,0.998867,31.229991,2.358601,2.859754,242.453320,81.356023,0.931261,2.980152,60604.0,1195.991200,0.532421,0.953410,4.858222,Position_6,siCTRL,R1,siCTRL_R1_Position_6,1,siCTRL_R1_Position_6_0,0.991730,-0.938351,-1.930080,0.123625,0.547243,0.836974,0.591119,-0.806584
0,ID3398,3398,1,85871.0,1183.610542,926.726637,0.0,0.0,0,165.315731,1,NaN,1.998253,2.0,0.0,2.0,171560.0,0.059087,0.878833,31.637928,2.586929,-2.215420,267.586902,113.759342,0.697861,2.352219,85857.5,1437.288260,0.522276,0.876916,4.905178,Position_6,siCTRL,R1,siCTRL_R1_Position_6,1,siCTRL_R1_Position_6_1,0.344727,-0.514774,-0.859501,0.426168,0.941168,0.337940,0.870404,-0.492338
0,ID3404,3404,2,13786.0,442.874015,1569.118383,0.0,0.0,0,66.253251,1,NaN,2.994565,3.0,0.0,3.0,41325.0,0.127577,0.606022,17.714423,-15.242835,23.264756,113.481620,63.315583,2.716260,1.792317,13790.0,537.780580,0.599189,0.930908,4.579552,Position_6,siCTRL,R1,siCTRL_R1_Position_6,1,siCTRL_R1_Position_6_2,0.452481,-0.634649,-1.087131,0.216251,0.899365,0.437198,0.805280,-0.592895
0,ID3392,3392,3,77274.0,165.415412,1456.679003,0.0,0.0,0,156.843182,1,NaN,0.996998,1.0,0.0,1.0,77052.0,0.054708,0.266454,7.668452,1.208453,1.235782,193.463786,128.476575,0.682642,1.505829,77282.5,1109.345584,0.789146,0.976770,3.990490,Position_6,siCTRL,R1,siCTRL_R1_Position_6,1,siCTRL_R1_Position_6_3,1.547278,-0.277612,-1.824890,0.063186,0.023516,0.999723,0.961713,-0.274060
0,ID3410,3410,4,83662.0,877.505105,310.833068,0.0,0.0,0,163.207867,1,NaN,3.997180,4.0,0.0,4.0,334500.0,0.106173,0.999677,37.641711,0.552070,1.260850,180.526636,147.336861,1.546721,1.225265,83682.0,1093.344279,0.879687,0.977902,3.779557,Position_6,siCTRL,R1,siCTRL_R1_Position_6,1,siCTRL_R1_Position_6_4,-0.381967,1.425093,1.807060,0.054790,0.927933,-0.372747,0.145189,0.989404


In [105]:
cmap = plt.get_cmap('hsv')
for time_point in range(len(df_temp.FRAME.unique())):
    plt.imshow(im[time_point,], cmap='gray')
    plt.quiver(x,y,u[time_point],v[time_point], color='yellow', pivot='mid', scale_units='xy', 
               scale=overlap, headaxislength=0, headlength=0, width=0.005, alpha=0.4)
    plt.plot(df_temp.loc[time_point,'POSITION_X'], df_temp.loc[time_point,'POSITION_Y'], 
             marker='o', linestyle='None', color=cmap(time_point*2))
    plt.axis('off')
    plt.savefig(output_image_folder+str(df_temp.File_name_raw.unique()[position])+'_AFT_tracks_'+str(time_point)+'.png', dpi=300, bbox_inches='tight')
    plt.close()      

In [89]:
cmap = plt.get_cmap('hsv')
for position in range(len(im_list_current)):

    # load current image
    im = io.imread(image_list[position])
    if single_frame == True:
        im = im[0,]

    # run AFT on all (relevant) frames
    x, y, u, v, im_theta, im_eccentricity = AFT.image_local_order(im, window_size, overlap,
                                                                  plot_overlay=False, 
                                                                  plot_angles=False,
                                                                  plot_eccentricity=False,
                                                                  save_figures=False)

    if single_frame == True:
        # display image and AFT
        plt.imshow(im, cmap='gray')
        plt.quiver(x,y,u[0],v[0], color='yellow', pivot='mid', scale_units='xy', 
                           scale=overlap, headaxislength=0, headlength=0, width=0.005, alpha=0.6)
    
    # get relevant part of subset_out df
    df_temp = df_subset_out.loc[df_subset_out.File_name_raw == im_list_current[position]]

    for trackID in range(len(df_subset_out.TRACK_ID.unique())):
    
        # create df for current track
        df_track = df_subset_out.loc[df_subset_out.TRACK_ID == df_subset_out.TRACK_ID.unique()[trackID]]
        df_track = df_track.sort_values(by='FRAME')
        df_track = df_track.reset_index(drop=True)
        df_track = df_track.drop_duplicates(subset=['FRAME'], ignore_index=True)

        if single_frame == True:
            for time_point in range(len(df_track)-1):
                plt.plot(df_track.loc[time_point:time_point+1,'POSITION_X'], df_track.loc[time_point:time_point+1,'POSITION_Y'], color='w')
                plt.plot(df_track.loc[time_point,'POSITION_X'], df_track.loc[time_point,'POSITION_Y'], marker='o', color=cmap(time_point*2))
                plt.axis('off')
                plt.savefig(output_folder+'AFT_tracks'+time_point+'.png', dpi=300, bbox_inches='tight')
                
        else:
            for time_point in range(len(df_track)-1):
                plt.imshow(im[time_point,], cmap='gray')
                plt.quiver(x,y,u[time_point],v[time_point], color='yellow', pivot='mid', scale_units='xy', 
                           scale=overlap, headaxislength=0, headlength=0, width=0.005, alpha=0.6)
                plt.plot(df_track.loc[time_point:time_point+1,'POSITION_X'], df_track.loc[time_point:time_point+1,'POSITION_Y'], color='w')
                plt.plot(df_track.loc[time_point,'POSITION_X'], df_track.loc[time_point,'POSITION_Y'], marker='o', color=cmap(time_point*2))
                plt.axis('off')
                plt.savefig(output_folder+'AFT_tracks'+time_point+'.png', dpi=300, bbox_inches='tight')

array(['Position_6', 'Position_12'], dtype=object)

In [ ]:
plt.figure(figsize=(18,12))
cmap = plt.get_cmap('jet')

if single_frame == True:
    plt.imshow(im, cmap='gray')
    plt.quiver(x,y,u[0],v[0], color='yellow', pivot='mid', scale_units='xy', scale=overlap, headaxislength=0, headlength=0, width=0.005, alpha=0.6)

    for time_point in range(len(df_track)-1):
        plt.plot(df_track.loc[time_point:time_point+1,'POSITION_X'], df_track.loc[time_point:time_point+1,'POSITION_Y'], color='w')
        plt.plot(df_track.loc[time_point,'POSITION_X'], df_track.loc[time_point,'POSITION_Y'], marker='o', color=cmap(time_point*2))
        plt.quiver(df_out_merge.loc[time_point,'POSITION_X'],df_out_merge.loc[time_point,'POSITION_Y'],
                   df_out_merge.loc[time_point,'track_angle_u'], df_out_merge.loc[time_point,'track_angle_v'], 
                   color=cmap(time_point*2), angles='xy', width=0.003, alpha=0.8)
else:
    for time_point in range(len(df_track)-1):
        plt.imshow(im[time_point,], cmap='gray')
        plt.quiver(x,y,u[time_point],v[time_point], color='yellow', pivot='mid', scale_units='xy', scale=overlap, headaxislength=0, headlength=0, width=0.005, alpha=0.6)
        plt.plot(df_track.loc[time_point:time_point+1,'POSITION_X'], df_track.loc[time_point:time_point+1,'POSITION_Y'], color='w')
        plt.plot(df_track.loc[time_point,'POSITION_X'], df_track.loc[time_point,'POSITION_Y'], marker='o', color=cmap(time_point*2))
        plt.quiver(df_out_merge.loc[time_point,'POSITION_X'],df_out_merge.loc[time_point,'POSITION_Y'],
                   df_out_merge.loc[time_point,'track_angle_u'], df_out_merge.loc[time_point,'track_angle_v'], 
                   color=cmap(time_point*2), angles='xy', width=0.003, alpha=0.8)

plt.axis('off')
plt.show()

In [44]:
df_subset.head()

,LABEL,ID,TRACK_ID,QUALITY,POSITION_X,POSITION_Y,POSITION_Z,POSITION_T,FRAME,RADIUS,...,PERIMETER,CIRCULARITY,SOLIDITY,SHAPE_INDEX,File_name_raw,Condition,experiment_nb,File_name,Repeat,Unique_ID
2289,ID1013263,1013263,2,104686.0,1385.149680,483.607060,0.0,207.0,207,182.519908,...,1591.645043,0.519144,0.868412,4.919954,Position_12,siCTRL,R2,siCTRL_R2_Position_12,2,siCTRL_R2_Position_12_2
2290,ID1001486,1001486,2,105146.0,1381.930468,489.474972,0.0,208.0,208,182.947118,...,1574.775161,0.532812,0.863993,4.856440,Position_12,siCTRL,R2,siCTRL_R2_Position_12,2,siCTRL_R2_Position_12_2
2291,ID822280,822280,2,97754.0,1404.447388,462.452799,0.0,68.0,68,176.396109,...,1374.348891,0.650345,0.901664,4.395751,Position_12,siCTRL,R2,siCTRL_R2_Position_12,2,siCTRL_R2_Position_12_2
2292,ID846344,846344,2,98672.0,1390.402170,462.590098,0.0,80.0,80,177.212568,...,1402.022846,0.630723,0.898931,4.463604,Position_12,siCTRL,R2,siCTRL_R2_Position_12,2,siCTRL_R2_Position_12_2
2293,ID1067028,1067028,2,126145.0,1385.398352,470.311650,0.0,250.0,250,200.409241,...,1827.181660,0.474932,0.842381,5.143859,Position_12,siCTRL,R2,siCTRL_R2_Position_12,2,siCTRL_R2_Position_12_2


In [ ]:
spots = 0 # in range(len(spots_list_current)):

# load current image
im = io.imread(im_list_current[spots])
if single_frame == True:
    im = im[0,]

# run AFT on all (relevant) frames
x, y, u, v, im_theta, im_eccentricity = AFT.image_local_order(im, window_size, overlap, save_path = input_folder+'output_data/',
                                                             plot_overlay=False, plot_angles=False, 
                                                             plot_eccentricity=False,
                                                             save_figures=False)

In [ ]:
AFT_coords = np.empty(shape=(len(x),2))
AFT_coords[:,0] = x
AFT_coords[:,1] = y

In [ ]:
# this needs to go in same for loop as before (spots)
df_subset = df_spots.loc[df_spots.File_name == spots_list_current[spots]]

In [ ]:
# this needs to go in same for loop as before (spots) + loop (tracks)
trackID = 3 # in range(len(df_subset.TRACK_ID.unique()))

df_track = df_subset.loc[df_subset.TRACK_ID == trackID]
df_track = df_track.sort_values(by='FRAME')
df_track = df_track.reset_index(drop=True)

In [ ]:
AFT_angle = []
track_angle = []
current_time_point = []

if single_frame == False:
    for time_point in range(len(df_track)-1):
        
        track_current = [df_track.loc[time_point, 'POSITION_X'], df_track.loc[time_point, 'POSITION_Y']]
        track_next = [df_track.loc[time_point+1, 'POSITION_X'], df_track.loc[time_point+1, 'POSITION_Y']]
        
        d_closest,idx_closest = spatial.KDTree(AFT_coords).query(track_current)
        AFT_angle = np.append(AFT_angle, np.ravel(im_theta[time_point])[idx_closest])
        
        track_length = [track_next[0]-track_current[0], track_next[1]-track_current[1]]
        track_norm = np.sqrt(track_length[0] ** 2 + track_length[1] ** 2)
        track_direction = [track_length[0]/track_norm, track_length[1]/track_norm]
        track_angle = np.append(track_angle, np.arctan2(track_direction[1], track_direction[0]))
    
        current_time_point = np.append(current_time_point, time_point)
else:
    for time_point in range(len(df_track)-1):
        
        track_current = [df_track.loc[time_point, 'POSITION_X'], df_track.loc[time_point, 'POSITION_Y']]
        track_next = [df_track.loc[time_point+1, 'POSITION_X'], df_track.loc[time_point+1, 'POSITION_Y']]
        
        d_closest,idx_closest = spatial.KDTree(AFT_coords).query(track_current)
        AFT_angle = np.append(AFT_angle, np.ravel(im_theta)[idx_closest])
        
        track_length = [track_next[0]-track_current[0], track_next[1]-track_current[1]]
        track_norm = np.sqrt(track_length[0] ** 2 + track_length[1] ** 2)
        track_direction = [track_length[0]/track_norm, track_length[1]/track_norm]
        track_angle = np.append(track_angle, np.arctan2(track_direction[1], track_direction[0]))
    
        current_time_point = np.append(current_time_point, time_point)
    
AFT_track_angle = AFT_angle-track_angle
AFT_track_angle_cos2 = np.cos(AFT_track_angle) ** 2

In [ ]:
df_out = pd.DataFrame(columns = ['FRAME','track_angle','AFT_angle','AFT_track_angle','AFT_track_angle_cos2',
                                'track_angle_u', 'track_angle_v','AFT_angle_u','AFT_angle_v'])

df_out.FRAME = current_time_point
df_out.track_angle = track_angle
df_out.AFT_angle = AFT_angle
df_out.AFT_track_angle = AFT_track_angle
df_out.AFT_track_angle_cos2 = AFT_track_angle_cos2

df_out.track_angle_u = np.cos(track_angle)
df_out.track_angle_v = np.sin(track_angle)
df_out.AFT_angle_u = np.cos(AFT_angle)
df_out.AFT_angle_v = np.sin(AFT_angle)

In [ ]:
df_out_merge = pd.merge(df_track, df_out, on='FRAME', how='outer')

In [ ]:
# to do: 
# check single_frame == True
# decide where to plot
# set up loops
# decide what to save in df_tracks
# check track vectors on current track (3)

### FOR all tracks in image

In [74]:
df_track_out = pd.DataFrame(columns = ['TRACK_ID','FRAME','track_angle','AFT_angle','AFT_track_angle','AFT_track_angle_cos2',
                                'track_angle_u', 'track_angle_v','AFT_angle_u','AFT_angle_v'])

for trackID in range(len(df_subset.TRACK_ID.unique())):

    # create df for current track
    df_track = df_subset.loc[df_subset.TRACK_ID == df_subset.TRACK_ID.unique()[trackID]]
    df_track = df_track.sort_values(by='FRAME')
    df_track = df_track.reset_index(drop=True)
    df_track = df_track.drop_duplicates(subset=['FRAME'], ignore_index=True)

    # initialise ouput arrays
    AFT_angle = []
    track_angle = []
    current_time_point = []

    # calculate angles for AFT and track
    if single_frame == False:
        for time_point in range(len(df_track)-1):
            
            track_current = [df_track.loc[time_point, 'POSITION_X'], df_track.loc[time_point, 'POSITION_Y']]
            track_next = [df_track.loc[time_point+1, 'POSITION_X'], df_track.loc[time_point+1, 'POSITION_Y']]
            
            d_closest,idx_closest = spatial.KDTree(AFT_coords).query(track_current)
            AFT_angle = np.append(AFT_angle, np.ravel(im_theta[time_point])[idx_closest])
            
            track_length = [track_next[0]-track_current[0], track_next[1]-track_current[1]]
            track_norm = np.sqrt(track_length[0] ** 2 + track_length[1] ** 2)
            track_direction = [track_length[0]/track_norm, track_length[1]/track_norm]
            track_angle = np.append(track_angle, np.arctan2(track_direction[1], track_direction[0]))
        
            current_time_point = np.append(current_time_point, time_point)
    else:
        for time_point in range(len(df_track)-1):
            
            track_current = [df_track.loc[time_point, 'POSITION_X'], df_track.loc[time_point, 'POSITION_Y']]
            track_next = [df_track.loc[time_point+1, 'POSITION_X'], df_track.loc[time_point+1, 'POSITION_Y']]
            
            d_closest,idx_closest = spatial.KDTree(AFT_coords).query(track_current)
            AFT_angle = np.append(AFT_angle, np.ravel(im_theta)[idx_closest])
            
            track_length = [track_next[0]-track_current[0], track_next[1]-track_current[1]]
            track_norm = np.sqrt(track_length[0] ** 2 + track_length[1] ** 2)
            track_direction = [track_length[0]/track_norm, track_length[1]/track_norm]
            track_angle = np.append(track_angle, np.arctan2(track_direction[1], track_direction[0]))
        
            current_time_point = np.append(current_time_point, time_point)

    # calculate angle difference and cosine squared of the angle difference
    AFT_track_angle = AFT_angle-track_angle
    AFT_track_angle_cos2 = np.cos(AFT_track_angle) ** 2

    df_out = pd.DataFrame(columns = ['TRACK_ID','FRAME','track_angle','AFT_angle','AFT_track_angle','AFT_track_angle_cos2',
                                'track_angle_u', 'track_angle_v','AFT_angle_u','AFT_angle_v'])

    df_out.TRACK_ID = np.full((len(df_track)-1, ), df_subset.TRACK_ID.unique()[trackID])
    df_out.FRAME = current_time_point
    df_out.track_angle = track_angle
    df_out.AFT_angle = AFT_angle
    df_out.AFT_track_angle = AFT_track_angle
    df_out.AFT_track_angle_cos2 = AFT_track_angle_cos2
    
    df_out.track_angle_u = np.cos(track_angle)
    df_out.track_angle_v = np.sin(track_angle)
    df_out.AFT_angle_u = np.cos(AFT_angle)
    df_out.AFT_angle_v = np.sin(AFT_angle)

    df_list = [df_track_out, df_out]

    df_track_out = pd.concat([df_track_out for df_track_out in df_list if not df_track_out.empty])
    

/var/folders/b8/814qyc5s5jx004x1r459c2gc0000gs/T/ipykernel_72169/1736063956.py:29: RuntimeWarning: invalid value encountered in scalar divide
  track_direction = [track_length[0]/track_norm, track_length[1]/track_norm]
/var/folders/b8/814qyc5s5jx004x1r459c2gc0000gs/T/ipykernel_72169/1736063956.py:29: RuntimeWarning: invalid value encountered in scalar divide
  track_direction = [track_length[0]/track_norm, track_length[1]/track_norm]
/var/folders/b8/814qyc5s5jx004x1r459c2gc0000gs/T/ipykernel_72169/1736063956.py:29: RuntimeWarning: invalid value encountered in scalar divide
  track_direction = [track_length[0]/track_norm, track_length[1]/track_norm]


In [78]:
df_track_out.track_angle.max()

np.float64(3.1220894568543196)

In [51]:
df_subset.TRACK_ID.unique()[trackID]

np.int64(2)

In [79]:
df_track_out.head()

,TRACK_ID,FRAME,track_angle,AFT_angle,AFT_track_angle,AFT_track_angle_cos2,track_angle_u,track_angle_v,AFT_angle_u,AFT_angle_v
0,2,0.0,2.234350,1.291034,-0.943316,0.344695,-0.615920,0.787808,0.276127,0.961121
1,2,1.0,-1.113907,1.248514,2.362421,0.506227,0.441159,-0.897429,0.316732,0.948515
2,2,2.0,1.860897,1.188163,-0.672733,0.611714,-0.286049,0.958215,0.373364,0.927685
3,2,3.0,2.212786,1.208582,-1.004203,0.288112,-0.598790,0.800906,0.354345,0.935115
4,2,4.0,2.120130,1.360158,-0.759971,0.525416,-0.522119,0.852873,0.209084,0.977898


In [55]:
pippo = df_spots.loc[df_spots.File_name_raw == 'Position_12']
pippo.TRACK_ID.unique()

array([2, 3, 4])

In [68]:
pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df_track

,LABEL,ID,TRACK_ID,QUALITY,POSITION_X,POSITION_Y,POSITION_Z,POSITION_T,FRAME,RADIUS,VISIBILITY,MANUAL_SPOT_COLOR,MEAN_INTENSITY_CH1,MEDIAN_INTENSITY_CH1,MIN_INTENSITY_CH1,MAX_INTENSITY_CH1,TOTAL_INTENSITY_CH1,STD_INTENSITY_CH1,CONTRAST_CH1,SNR_CH1,ELLIPSE_X0,ELLIPSE_Y0,ELLIPSE_MAJOR,ELLIPSE_MINOR,ELLIPSE_THETA,ELLIPSE_ASPECTRATIO,AREA,PERIMETER,CIRCULARITY,SOLIDITY,SHAPE_INDEX,File_name_raw,Condition,experiment_nb,File_name,Repeat,Unique_ID
0,ID777586,777586,3,53017.0,396.110002,758.896040,0.0,0.0,0,129.906413,1,NaN,1.998604,2.0,1.0,2.0,105956.0,0.037335,0.776897,46.810224,-1.399351,-3.007568,158.018501,108.751923,1.568579,1.453018,53016.5,952.266547,0.734690,0.947163,4.135738,Position_12,siCTRL,R2,siCTRL_R2_Position_12,2,siCTRL_R2_Position_12_3
1,ID782737,782737,3,46026.0,397.510809,754.592967,0.0,1.0,1,121.059756,1,NaN,1.998740,2.0,1.0,2.0,92020.0,0.035472,0.809471,50.414340,-0.902530,1.241759,142.636407,104.828486,1.421051,1.360665,46041.5,830.527457,0.838786,0.969019,3.870609,Position_12,siCTRL,R2,siCTRL_R2_Position_12,2,siCTRL_R2_Position_12_3
2,ID778787,778787,3,47130.0,397.528330,754.624376,0.0,1.0,1,122.518805,1,NaN,1.974344,2.0,0.0,2.0,93114.0,0.170000,0.805116,10.359930,-0.777196,1.548893,144.757787,106.140573,1.397304,1.363831,47158.0,841.332413,0.837202,0.971369,3.874270,Position_12,siCTRL,R2,siCTRL_R2_Position_12,2,siCTRL_R2_Position_12_3
3,ID777560,777560,3,40341.0,398.281962,745.063328,0.0,2.0,2,113.316463,1,NaN,1.974217,2.0,0.0,2.0,79632.0,0.167021,0.810413,10.582361,-0.385746,0.868741,124.846198,105.256799,1.298656,1.186111,40340.0,756.148579,0.886608,0.981341,3.764776,Position_12,siCTRL,R2,siCTRL_R2_Position_12,2,siCTRL_R2_Position_12_3
4,ID777591,777591,3,39339.0,398.264724,745.050621,0.0,2.0,2,111.904554,1,NaN,1.999085,2.0,1.0,2.0,78636.0,0.030239,0.812940,59.288922,-0.313241,0.809448,123.157746,102.981060,1.282759,1.195926,39341.0,745.600589,0.889289,0.982052,3.759097,Position_12,siCTRL,R2,siCTRL_R2_Position_12,2,siCTRL_R2_Position_12_3
5,ID777555,777555,3,39110.0,396.834092,749.878452,0.0,3.0,3,111.594075,1,NaN,1.974515,2.0,0.0,2.0,77245.0,0.166891,0.833534,10.757027,-0.578257,-0.543759,123.580535,100.530346,1.442676,1.229286,39123.0,739.032218,0.900151,0.990080,3.736348,Position_12,siCTRL,R2,siCTRL_R2_Position_12,2,siCTRL_R2_Position_12_3
6,ID777613,777613,3,38193.0,396.897224,749.862121,0.0,3.0,3,110.259736,1,NaN,1.998717,2.0,1.0,2.0,76331.0,0.035797,0.834530,50.798188,-0.002788,-0.427091,122.045683,99.611571,1.408745,1.225216,38193.0,734.288758,0.890143,0.988790,3.757292,Position_12,siCTRL,R2,siCTRL_R2_Position_12,2,siCTRL_R2_Position_12_3
7,ID777557,777557,3,39939.0,391.924081,740.029932,0.0,4.0,4,112.761725,1,NaN,1.973039,2.0,0.0,2.0,78817.0,0.172015,0.823175,10.357720,-0.406517,-0.260616,121.140409,105.686695,1.506912,1.146222,39946.0,750.930767,0.890191,0.983250,3.757191,Position_12,siCTRL,R2,siCTRL_R2_Position_12,2,siCTRL_R2_Position_12_3
8,ID777587,777587,3,38939.0,391.947117,740.097737,0.0,4.0,4,111.350644,1,NaN,1.998332,2.0,1.0,2.0,77857.0,0.040812,0.824560,44.256513,0.041297,0.211627,119.385085,103.986920,1.470769,1.148078,38952.5,739.292864,0.895596,0.983773,3.745837,Position_12,siCTRL,R2,siCTRL_R2_Position_12,2,siCTRL_R2_Position_12_3
9,ID777588,777588,3,39484.0,390.528948,737.814985,0.0,5.0,5,112.109169,1,NaN,1.995188,2.0,1.0,2.0,78774.0,0.069205,0.832230,26.190382,-0.354741,-2.076215,127.056645,100.880943,1.263091,1.259471,39485.0,774.979356,0.826155,0.966171,3.900085,Position_12,siCTRL,R2,siCTRL_R2_Position_12,2,siCTRL_R2_Position_12_3


In [72]:
pippo = df_track.drop_duplicates(subset=['FRAME'], ignore_index=True)
len(pippo)

312

In [73]:
pippo.head(30)

,LABEL,ID,TRACK_ID,QUALITY,POSITION_X,POSITION_Y,POSITION_Z,POSITION_T,FRAME,RADIUS,VISIBILITY,MANUAL_SPOT_COLOR,MEAN_INTENSITY_CH1,MEDIAN_INTENSITY_CH1,MIN_INTENSITY_CH1,MAX_INTENSITY_CH1,TOTAL_INTENSITY_CH1,STD_INTENSITY_CH1,CONTRAST_CH1,SNR_CH1,ELLIPSE_X0,ELLIPSE_Y0,ELLIPSE_MAJOR,ELLIPSE_MINOR,ELLIPSE_THETA,ELLIPSE_ASPECTRATIO,AREA,PERIMETER,CIRCULARITY,SOLIDITY,SHAPE_INDEX,File_name_raw,Condition,experiment_nb,File_name,Repeat,Unique_ID
0,ID777586,777586,3,53017.0,396.110002,758.896040,0.0,0.0,0,129.906413,1,NaN,1.998604,2.0,1.0,2.0,105956.0,0.037335,0.776897,46.810224,-1.399351,-3.007568,158.018501,108.751923,1.568579,1.453018,53016.5,952.266547,0.734690,0.947163,4.135738,Position_12,siCTRL,R2,siCTRL_R2_Position_12,2,siCTRL_R2_Position_12_3
1,ID782737,782737,3,46026.0,397.510809,754.592967,0.0,1.0,1,121.059756,1,NaN,1.998740,2.0,1.0,2.0,92020.0,0.035472,0.809471,50.414340,-0.902530,1.241759,142.636407,104.828486,1.421051,1.360665,46041.5,830.527457,0.838786,0.969019,3.870609,Position_12,siCTRL,R2,siCTRL_R2_Position_12,2,siCTRL_R2_Position_12_3
2,ID777560,777560,3,40341.0,398.281962,745.063328,0.0,2.0,2,113.316463,1,NaN,1.974217,2.0,0.0,2.0,79632.0,0.167021,0.810413,10.582361,-0.385746,0.868741,124.846198,105.256799,1.298656,1.186111,40340.0,756.148579,0.886608,0.981341,3.764776,Position_12,siCTRL,R2,siCTRL_R2_Position_12,2,siCTRL_R2_Position_12_3
3,ID777555,777555,3,39110.0,396.834092,749.878452,0.0,3.0,3,111.594075,1,NaN,1.974515,2.0,0.0,2.0,77245.0,0.166891,0.833534,10.757027,-0.578257,-0.543759,123.580535,100.530346,1.442676,1.229286,39123.0,739.032218,0.900151,0.990080,3.736348,Position_12,siCTRL,R2,siCTRL_R2_Position_12,2,siCTRL_R2_Position_12_3
4,ID777557,777557,3,39939.0,391.924081,740.029932,0.0,4.0,4,112.761725,1,NaN,1.973039,2.0,0.0,2.0,78817.0,0.172015,0.823175,10.357720,-0.406517,-0.260616,121.140409,105.686695,1.506912,1.146222,39946.0,750.930767,0.890191,0.983250,3.757191,Position_12,siCTRL,R2,siCTRL_R2_Position_12,2,siCTRL_R2_Position_12_3
5,ID777588,777588,3,39484.0,390.528948,737.814985,0.0,5.0,5,112.109169,1,NaN,1.995188,2.0,1.0,2.0,78774.0,0.069205,0.832230,26.190382,-0.354741,-2.076215,127.056645,100.880943,1.263091,1.259471,39485.0,774.979356,0.826155,0.966171,3.900085,Position_12,siCTRL,R2,siCTRL_R2_Position_12,2,siCTRL_R2_Position_12_3
6,ID777606,777606,3,40996.0,390.156044,736.677673,0.0,6.0,6,114.236201,1,NaN,1.998902,2.0,1.0,2.0,81947.0,0.033113,0.799312,53.632634,-1.406415,-1.395328,125.645683,105.329925,1.337487,1.192877,40997.5,783.925914,0.838334,0.967538,3.871652,Position_12,siCTRL,R2,siCTRL_R2_Position_12,2,siCTRL_R2_Position_12_3
7,ID777547,777547,3,42269.0,390.926616,743.918745,0.0,7.0,7,115.987280,1,NaN,1.975157,2.0,0.0,2.0,83480.0,0.162782,0.800969,10.792795,0.512654,-0.325569,126.682571,106.452745,1.400947,1.190036,42264.0,765.413422,0.906543,0.989476,3.723152,Position_12,siCTRL,R2,siCTRL_R2_Position_12,2,siCTRL_R2_Position_12_3
8,ID777603,777603,3,42834.0,394.166507,746.262403,0.0,8.0,8,116.777023,1,NaN,1.998716,2.0,1.0,2.0,85599.0,0.035814,0.745862,47.684909,0.165827,-0.265948,126.480218,107.039418,-1.516203,1.181623,42841.5,772.277076,0.902668,0.991105,3.731134,Position_12,siCTRL,R2,siCTRL_R2_Position_12,2,siCTRL_R2_Position_12_3
9,ID778771,778771,3,47467.0,416.735453,746.377239,0.0,9.0,9,122.986859,1,NaN,1.908041,2.0,0.0,2.0,90672.0,0.308826,0.790776,5.456528,262.821673,-30.899023,412.180159,205.879340,2.401989,2.002047,47519.0,5852.280708,0.017435,0.134914,26.846737,Position_12,siCTRL,R2,siCTRL_R2_Position_12,2,siCTRL_R2_Position_12_3


In [ ]:
plt.figure(figsize=(18,12))
cmap = plt.get_cmap('jet')

if single_frame == True:
    plt.imshow(im, cmap='gray')
    plt.quiver(x,y,u[0],v[0], color='yellow', pivot='mid', scale_units='xy', scale=overlap, headaxislength=0, headlength=0, width=0.005, alpha=0.6)

    for time_point in range(len(df_track)-1):
        plt.plot(df_track.loc[time_point:time_point+1,'POSITION_X'], df_track.loc[time_point:time_point+1,'POSITION_Y'], color='w')
        plt.plot(df_track.loc[time_point,'POSITION_X'], df_track.loc[time_point,'POSITION_Y'], marker='o', color=cmap(time_point*2))
        plt.quiver(df_out_merge.loc[time_point,'POSITION_X'],df_out_merge.loc[time_point,'POSITION_Y'],
                   df_out_merge.loc[time_point,'track_angle_u'], df_out_merge.loc[time_point,'track_angle_v'], 
                   color=cmap(time_point*2), angles='xy', width=0.003, alpha=0.8)
else:
    for time_point in range(len(df_track)-1):
        plt.imshow(im[time_point,], cmap='gray')
        plt.quiver(x,y,u[time_point],v[time_point], color='yellow', pivot='mid', scale_units='xy', scale=overlap, headaxislength=0, headlength=0, width=0.005, alpha=0.6)
        plt.plot(df_track.loc[time_point:time_point+1,'POSITION_X'], df_track.loc[time_point:time_point+1,'POSITION_Y'], color='w')
        plt.plot(df_track.loc[time_point,'POSITION_X'], df_track.loc[time_point,'POSITION_Y'], marker='o', color=cmap(time_point*2))
        plt.quiver(df_out_merge.loc[time_point,'POSITION_X'],df_out_merge.loc[time_point,'POSITION_Y'],
                   df_out_merge.loc[time_point,'track_angle_u'], df_out_merge.loc[time_point,'track_angle_v'], 
                   color=cmap(time_point*2), angles='xy', width=0.003, alpha=0.8)

plt.axis('off')
plt.show()

In [ ]:
cmap = plt.get_cmap('jet')
for time_point in range(len(df_track)-1):
    plt.plot(df_track.loc[time_point:time_point+1,'POSITION_X'], df_track.loc[time_point:time_point+1,'POSITION_Y'], color='k')
    plt.plot(df_track.loc[time_point,'POSITION_X'], df_track.loc[time_point,'POSITION_Y'], marker='o', color=cmap(time_point*2))
    plt.quiver(df_out_merge.loc[time_point,'POSITION_X'],df_out_merge.loc[time_point,'POSITION_Y'],
               df_out_merge.loc[time_point,'track_angle_u'], df_out_merge.loc[time_point,'track_angle_v'], 
               color=cmap(time_point*2), angles='xy', width=0.003, alpha=0.8)

plt.axis('equal')
plt.axis('off')

plt.gca().invert_yaxis()
plt.show()

In [ ]:
cmap = plt.get_cmap('jet')
for time_point in range(len(df_track)-1):
    plt.plot(df_track.loc[time_point:time_point+1,'POSITION_X'], df_track.loc[time_point:time_point+1,'POSITION_Y'], color='k')
    plt.plot(df_track.loc[time_point,'POSITION_X'], df_track.loc[time_point,'POSITION_Y'], marker='.', color=cmap(time_point*2), markersize = 7)

plt.quiver(df_out_merge.POSITION_X,df_out_merge.POSITION_Y,df_out_merge.AFT_angle_u, df_out_merge.AFT_angle_v, color='y', 
           pivot='mid', headaxislength=0, headlength=0, width=0.005)

plt.axis('equal')
plt.axis('off')

plt.gca().invert_yaxis()
plt.show()